In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt



import magic_input_processing_LAE as magic

from importlib.machinery import SourceFileLoader

wr = SourceFileLoader("mobius", r"../mobius.py").load_module()
cu = SourceFileLoader("mobius_calib_uncert_lmfit", r"../mobius_calib_uncert_lmfit.py").load_module()

import magicopt

In [2]:
# Make an empty dataframe that will later contain your input data.
# The range provided is the years (inclusive) that you want in your input data.
df = magic.empty_input_df(1850, 2100)
scenar = "med85b"          # Name of scenario, min, med or max, 4.5 or 8.5, nothing or b ()
deltaP = 1.08              # Relative increase in Precipitation - ratio - from 1970-2000 (nothing) or 2000-2022 (b) to 2070-2100
deltaT = 2.17              # Relative increase in Temperature - ratio - from 1970-2000 (nothing) or 2000-2022 (b) to 2070-2100


# Use the pandas library to read in data from various locations. You need them in a pandas dataframe form.
# You can e.g. use pd.read_excel or pd.read_csv depending on your source.

# Right now precipitation and air temperature has to be daily values (I will maybe make it accept a source that is monthly later)
# The columns can be called whatever you want.

met_df = pd.read_excel('example_Langtjern_best.xls', sheet_name='met_data', index_col='Date', parse_dates=True)

met_df




,T,P,E
Date,,,
1957-01-01,-5.053226,25.70,0.02
1957-02-01,-8.547857,42.48,0.00
1957-03-01,-5.432581,23.34,0.00
1957-04-01,0.795333,17.70,0.90
1957-05-01,4.996452,52.44,24.60
...,...,...,...
2022-08-01,13.220968,68.54,60.78
2022-09-01,7.863333,102.32,51.98
2022-10-01,3.736452,72.38,37.78


In [3]:
# Example of a df that can be used to patch the deposition series. The values are relative to a reference year that you provide
# yourself below, here it is 1989-1991 (also for the future), so I had to calculate the 2017-2021 / 1989-1991 ratio.
#           SO4 (in P)     NH4 (in P)       NO3 (in P)
#2017-2021    7.585         14.169            15.537
#1989-1991   42.993          41.8             38.271
#
# Ratio      0.1764         0.3390            0.4060



emep_df1 = magic.get_emep_deposition_scales()

emep_df = emep_df1.drop(columns=['Ca'])
emep_df

# This is to adjust future deposition according to future trends in precipitation
emep_df["SO4"][-1] = emep_df["SO4"][-1]/ (deltaP*100/96.2)
emep_df["NH4"][-1] = emep_df["NH4"][-1]/ (deltaP*100/96.2)
emep_df["NO3"][-1] = emep_df["NO3"][-1]/ (deltaP*100/96.2)
emep_df["SO4"][-2] = emep_df["SO4"][-2]/ (1+((deltaP-1) * 50 / 100)*100/96.2)
emep_df["NH4"][-2] = emep_df["NH4"][-2]/ (1+((deltaP-1) * 50 / 100)*100/96.2)
emep_df["NO3"][-2] = emep_df["NO3"][-2]/ (1+((deltaP-1) * 50 / 100)*100/96.2)
emep_df["SO4"][-3] = emep_df["SO4"][-3]/ (1+((deltaP-1) * 30 / 100)*100/96.2)
emep_df["NH4"][-3] = emep_df["NH4"][-3]/ (1+((deltaP-1) * 30 / 100)*100/96.2)
emep_df["NO3"][-3] = emep_df["NO3"][-3]/ (1+((deltaP-1) * 30 / 100)*100/96.2)

In [4]:
# Example of a df that can be used to patch the deposition series in the future. The values are relative to a reference year that you provide
# yourself below.

#emep_df_future = magic.get_emep_deposition_scales_future()

emep_df

,SO4,NH4,NO3
Date,,,
1850-01-01,0.050000,0.000000,0.000000
1915-01-01,0.610000,NaN,NaN
1925-01-01,0.640000,NaN,NaN
1940-01-01,0.780000,NaN,NaN
1955-01-01,1.200000,NaN,NaN
1960-01-01,1.260000,0.990000,0.530000
1965-01-01,1.480000,1.000000,0.630000
1970-01-01,1.600000,1.020000,0.740000
1975-01-01,1.410000,1.120000,0.760000


In [5]:
# Similarly, read in runoff data from excel. This does not have to be in a separate document from the precip and air temp.
# That is just how I organized this example.

runoff_df = pd.read_excel('example_Langtjern_best.xls', sheet_name='runoff_data', index_col='Date', parse_dates=True)

runoff_df

,Q
Date,
1850-01-01,50.000000
1850-02-01,NaN
1850-03-01,NaN
1850-04-01,NaN
1850-05-01,NaN
...,...
2022-08-01,3.048098
2022-09-01,95.133655
2022-10-01,67.631489


In [6]:
# Read in deposition data (concentration in precipitation). These can be daily, monthly or yearly in the source.
# The columns should include SO4, NO3, NH4, Ca, and one of Mg or Cl (depending on what you use for sea salt correction)

dep_df = pd.read_excel('example_Langtjern_best.xls', sheet_name='dep_data', index_col='Date', parse_dates=True)

dep_df

,Ca,Mg,Na,K,NH4,SO4,Cl,NO3
Date,,,,,,,,
1974-01-01,6.483791,3.292181,NaN,NaN,20.000000,50.467290,NaN,27.142857
1974-02-01,6.483791,3.292181,NaN,NaN,20.000000,50.467290,NaN,27.142857
1974-03-01,6.483791,3.292181,NaN,NaN,20.000000,50.467290,NaN,27.142857
1974-04-01,6.483791,3.292181,NaN,NaN,20.000000,50.467290,NaN,27.142857
1974-05-01,6.483791,3.292181,NaN,NaN,20.000000,50.467290,NaN,27.142857
...,...,...,...,...,...,...,...,...
2022-08-01,3.750623,1.868313,8.873913,1.846547,6.985714,4.834891,9.11831,11.128571
2022-09-01,3.750623,1.868313,8.873913,1.846547,6.985714,4.834891,9.11831,11.128571
2022-10-01,3.750623,1.868313,8.873913,1.846547,6.985714,4.834891,9.11831,11.128571


In [7]:
# Read in observed runoff concentrations. You should at least have SO4
# and one of Cl or Mg (depending on which one you use for sea salt correction).

runoff_chem_df = pd.read_excel('example_Langtjern_best.xls', sheet_name='runoff_chem_data', index_col='Date', parse_dates=True)

runoff_chem_df

,Ca,Cl,K,Mg,Na,NH4,NO3,SO4,pH,OA,Exc. Ca soil,Exc. Mg soil,Exc. Na soil,Exc. K soil,Total base saturation soil,H,Aln+,ANC
Date,,,,,,,,,,,,,,,,,,
1974-01-01,82.514085,17.541334,2.889541,24.679170,40.452898,2.253751,1.317426,82.828974,5.000,NaN,NaN,NaN,NaN,NaN,NaN,10.000000,NaN,NaN
1974-02-01,82.871176,17.582239,3.141717,24.619752,35.333029,0.950532,1.273259,80.924335,5.010,NaN,NaN,NaN,NaN,NaN,NaN,9.772372,NaN,NaN
1974-03-01,87.769686,18.216440,3.715672,24.496212,34.908394,1.292530,1.895863,86.054958,4.940,NaN,NaN,NaN,NaN,NaN,NaN,11.481536,NaN,NaN
1974-04-01,72.699965,18.833712,5.325798,24.178131,34.511299,1.827422,4.027245,89.702915,4.535,NaN,NaN,NaN,NaN,NaN,NaN,29.174270,NaN,NaN
1974-05-01,55.423837,11.518756,3.542142,17.713381,21.551758,0.813796,1.027134,66.415459,4.980,NaN,NaN,NaN,NaN,NaN,NaN,10.471285,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-08-01,49.663109,13.242520,2.717573,11.688721,28.573366,1.347087,0.145228,13.871101,5.530,32.047525,NaN,NaN,NaN,NaN,NaN,2.951209,1.778975,65.43175
2022-09-01,50.205550,15.163160,4.200654,12.938793,29.354306,4.022642,0.266716,14.203655,5.460,34.892475,NaN,NaN,NaN,NaN,NaN,3.467369,2.879398,67.50000
2022-10-01,50.056323,14.297651,3.228295,12.901691,28.127381,1.112147,0.363695,14.141303,4.855,49.200900,NaN,NaN,NaN,NaN,NaN,13.963684,2.821333,66.50000


In [8]:

# Add the precipitation time series to the input data. The patch ref range is the range that it uses for computing
# averages that it uses to patch in missing values. The series you provide has to be without holes from its own start to end.
magic.add_precip_ts(df, met_df['P'], patch_ref_range=(1971, 2000), present=(2022), delta=deltaP)
magic.add_precip_ref_ts(df, met_df['P'], patch_ref_range=(1971, 2000), present=(2022), delta=1.00)
magic.add_pet_ts(df, met_df['E'], patch_ref_range=(1971, 2000), present=(2022), delta=1.0)

# Add a runoff series.
#NOTE: The Runoff series should not be patched at all. It will only be used for reference in the deposition correction
#      and for calibrating the eventual hydr. module.
magic.add_runoff_ts(df, runoff_df['Q'])

# Add in air temperature series similarly to precipitation.
magic.add_air_temperature_ts(df, met_df['T'], patch_ref_range=(1971, 2000), present =(2022), delta =deltaT)


# Add in sea salt corrected deposition.
# Before adding this you have to add Precipitation, Runoff series to your input dataframe.
# The runoff series has to be valid for the sea_salt_ref_range. The precipitation and measured deposition series should be
# valid for the patch_ref_range and the ca_ref_range. Note that measured deposition series are linearly interpolated between
# valid values, so it is ok with a few holes.

'''
df  : The dataframe you want to write the inputs to. It must already have a 'Precipitation' and a 'Runoff' column
dep_df : A date-indexed dataframe with observed concentrations in precipitation, with columns ['Ca', 'Mg', 'Na', 'K', 'NH4', 'SO4', 'Cl', 'NO3', 'F']   F is optional
runoff_chem_df : A date-indexed dataframe with observed concentrations in runoff, with the same columns as source_df.
source_is_mg_l : True if the unit of the source_df is mg/l, False if they are meq/m3=µeq/l
runoff_is_mg_l : True if the unit of the runoff_df is mg/l, False if they are meq/m3=µeq/l
n_dep_given_as_n : Only relevant if source is mg/l: True if the mass of NO3 and NH4 deposition is given as the mass of N, False if it is given as the mass of the full molecule.
s_dep_given_as_s : Only relevant if source is mg/l: True if the mass of SO4 deposition is given as the mass of S, False if it is given as the mass of the full molecule.
sea_salt_ref : The reference deposition series for sea salt correction. Usually 'Cl', but can also use 'Mg'.
sea_salt_ref_range : pair (start_year, end_year). This is the range that is used to compute the sea_salt_factor and the SO4* factor.
patch_scale : A date-indexed dataframe containing some of the columns of the input_df. Should have some values of historical deposition relative to a reference year. Example : get_emep_deposition_scales()
patch_ref_range : pair (start_year, end_year). Range of years to compute the value for the reference value that scales the values used for patching.
so4_weathering : how much of the so4 runoff should be accounted for by weathering and not deposition
ca_ref_range : If not None, compute Ca excess deposition relative to SO4 excess deposition with reference to this range.
ca_excess_weight : Ca excess deposition relative to SO4 excess deposition is multiplied with this value
n_dry_deposition_factor : Multiply NH4 and NO3 deposition by this value
'''

magic.add_deposition_ts(df, dep_df, runoff_chem_df, sea_salt_ref='Cl', sea_salt_ref_range=(1981,2000), 
                        source_is_mg_l=False, runoff_is_mg_l=False,
                        patch_scale=emep_df, patch_ref_range=(1989,1991),
                        so4_weathering=0.0,
                        ca_ref_range=(1974,2002), ca_excess_weight=0.0, 
                        n_dry_deposition_factor=1.14)

df

Reference value for Precipitation is 
938.8752380952379
Reference value for Precipitation_ref is 
938.8752380952379
Reference value for Potential evapotranspiration is 
513.3933333333332
Reference value for Air temperature is 
2.997855030616893
The sea salt factor was: 1.11398
The SO4* factor was 0.980422
scale for SO4 is : 42.992692
The Ca* factor was 0.08
Reference year for Ca is: 1974


,Precipitation,Precipitation_ref,Potential evapotranspiration,Runoff,Air temperature,SO4 conc in precipitation,Cl conc in precipitation,Ca conc in precipitation,NH4 conc in precipitation,NO3 conc in precipitation,...,Observed runoff conc pH,Observed runoff conc OA,Observed runoff conc Exc. Ca soil,Observed runoff conc Exc. Mg soil,Observed runoff conc Exc. Na soil,Observed runoff conc Exc. K soil,Observed runoff conc Total base saturation soil,Observed runoff conc H,Observed runoff conc Aln+,Observed runoff conc ANC
1850-01-01,74.023158,74.023158,0.045112,50.0,-6.297248,2.149635,8.244876,0.411177,0.000000,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1850-02-01,52.793370,52.793370,0.112780,NaN,-6.073307,2.180501,8.249285,0.411177,0.031350,0.015367,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1850-03-01,69.982958,69.982958,0.192143,NaN,-3.104538,2.211368,8.253695,0.411177,0.062700,0.030733,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1850-04-01,45.799128,45.799128,4.161989,NaN,1.408988,2.242234,8.258104,0.411177,0.094050,0.046100,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1850-05-01,61.907148,61.907148,60.084022,NaN,7.675182,2.273101,8.262514,0.411177,0.125400,0.061466,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2100-08-01,101.540718,92.806032,97.427476,NaN,16.494717,3.124515,8.384144,0.411177,11.907283,5.096366,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2100-09-01,103.627085,94.712927,59.971242,NaN,11.681870,3.124515,8.384144,0.411177,11.907283,5.096366,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2100-10-01,120.675374,110.294697,23.396362,NaN,7.104200,3.124515,8.384144,0.411177,11.907283,5.096366,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2100-11-01,110.733119,101.207689,2.193774,NaN,1.880559,3.124515,8.384144,0.411177,11.907283,5.096366,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
# Plot the series that you have added to your input dataframe

# magic.plot_inputs(df)

In [10]:
# Export the input dataframe to a Mobius input file (.dat or .xlsx) or to a .csv file.

magic.write_as_input_file(df, 'LAE_inputs.xlsx')

In [11]:
# Initializing MAGIC for Langtjern
wr.initialize('../../Applications/MAGIC/magic_forest.dll')

parfile = './langtjern_parameters_Best_SO4Control_addCl_new.dat'
infile  = './LAE_inputs.xlsx'

dataset = wr.DataSet.setup_from_parameter_and_input_files(parfile, infile)

#Running MAGIC with no weatherting increase
weaIn = 0
dataset.set_parameter_double("Yearly weathering increase", ['Soil1'], weaIn)

dataset.run_model()  # to have some initial result values for comparison

# Defining indexes for post-processing
idx = ('Soil1', 'Stream')

In [12]:
# res_list = [
#     ["Runoff",[]],
#     ["Ca(2+) ionic concentration",["Stream"]],
#     ["Mg(2+) ionic concentration",["Stream"]],
#     ["Na(+) ionic concentration",["Stream"]],
#     ["K(+) ionic concentration",["Stream"]],
#     ["SO4(2-) ionic concentration",["Stream"]],
#     ["Cl(-) ionic concentration",["Stream"]],
#     ["NO3(-) ionic concentration",["Stream"]],
#     ["H(+) ionic concentration",["Stream"]],
#     ["pH",["Stream"]],
#     ["HCO3 (Bicarbonate) ionic concentration",["Stream"]],
#     ["Charge balance alkalinity",["Stream"]],
#     ["Al(3+) ionic concentration",["Stream"]],
#     ["Total aluminum in solution (ionic + SO4-F-DOC complexes)",["Stream"]],
#     ["Aluminum in solution as organic complexes (AlA, Al(HA)(+)",["Stream"]],
#     ["Organic acid concentration",["Stream"]],
#     ["Exchangeable Ca on soil as % of CEC",["Soil1"]],
#     ["Exchangeable Mg on soil as % of CEC",["Soil1"]],
#     ["Exchangeable Na on soil as % of CEC",["Soil1"]],
#     ["Exchangeable K on soil as % of CEC",["Soil1"]],
#     ["Base saturation of soil (ECa + EMg + ENa + EK)",["Soil1"]]
#     ]
# results = cu.get_result_dataframe(dataset, res_list)
# results
# #results.to_csv('Results_test.csv')

In [13]:
#params, comparisons = magicopt.get_base_cations_setup(dataset, idx)

#params

#cu.print_goodness_of_fit(dataset, comparisons)
#cu.plot_objective(dataset, comparisons)

In [19]:
#Running MAGIC with no weatherting increase
weaIn = 0
dataset.set_parameter_double("Yearly weathering increase", ['Soil1'], weaIn)
dataset.run_model()

# List of results to export
res_list = [
        ["SO4 total deposition",["Soil1"]],
        ["NO3 total deposition",["Soil1"]],
        ["NH4 total deposition",["Soil1"]],
        ["Cl total deposition",["Soil1"]],
        ["Ca total deposition",["Soil1"]],
        ["Mg total deposition",["Soil1"]],
        ["K total deposition",["Soil1"]],
        ["Na total deposition",["Soil1"]],
        ["Effective precipitation",[]],
        ["Runoff",[]],
        ["Ca(2+) ionic concentration",["Stream"]],
        ["Mg(2+) ionic concentration",["Stream"]],
        ["Na(+) ionic concentration",["Stream"]],
        ["K(+) ionic concentration",["Stream"]],
        ["Sum of base cation concentrations (Ca + Mg + Na + K)",["Stream"]],
        ["SO4(2-) ionic concentration",["Stream"]],
        ["Cl(-) ionic concentration",["Stream"]],
        ["NO3(-) ionic concentration",["Stream"]],
        ["Sum of acid anion concentrations (SO4 + Cl + NO3 + F)",["Stream"]],
        ["H(+) ionic concentration",["Stream"]],
        ["pH",["Stream"]],
        ["HCO3 (Bicarbonate) ionic concentration",["Stream"]],
        ["Charge balance alkalinity",["Stream"]],
        ["Al(3+) ionic concentration",["Stream"]],
        ["Total aluminum in solution (ionic + SO4-F-DOC complexes)",["Stream"]],
        ["Total Al(3+) charges",["Stream"]],
        ["Aluminum in solution as organic complexes (AlA, Al(HA)(+)",["Stream"]],
        ["Total anionic charge from DOC (triprotic acid) in solution (H2AM, HA2M, A3M)",["Stream"]],
        ["Organic acid concentration",["Stream"]],
        ["Exchangeable Ca on soil as % of CEC",["Soil1"]],
        ["Exchangeable Mg on soil as % of CEC",["Soil1"]],
        ["Exchangeable Na on soil as % of CEC",["Soil1"]],
        ["Exchangeable K on soil as % of CEC",["Soil1"]],
        ["Base saturation of soil (ECa + EMg + ENa + EK)",["Soil1"]]
        ]
# Exporting results to dataframe
results = cu.get_result_dataframe(dataset, res_list)

# Defining some date masks for dep_S and printing the average dep_S over those periods
past_mask_d = (results.index < '1861-01-01')
ref_mask_d = (results.index > '2050-12-01') & (results.index < '2061-01-01')
fut_mask_d = (results.index > '2089-12-01') & (results.index < '2101-01-01')

dep_S = results.loc[:,'SO4 total deposition [Soil1]']

mean_depS_past = np.mean(dep_S.loc[past_mask_d])
mean_depS_ref = np.mean(dep_S.loc[ref_mask_d])
mean_depS_fut = np.mean(dep_S.loc[fut_mask_d])

print('Pre-industrial dep_S is: ',mean_depS_past)
print('2050-2060 dep_S is: ',mean_depS_ref)
print('2090-2100 dep_S is: ',mean_depS_fut)

Pre-industrial dep_S is:  0.327642366450659
2050-2060 dep_S is:  0.2717610494767462
2090-2100 dep_S is:  0.26787107365505397


In [16]:
#Running MAGIC with no weatherting increase
weaIn = 0
dataset.set_parameter_double("Yearly weathering increase", ['Soil1'], weaIn)
dataset.run_model()

# List of results to export
res_list = [
        ["SO4 total deposition",["Soil1"]],
        ["NO3 total deposition",["Soil1"]],
        ["NH4 total deposition",["Soil1"]],
        ["Cl total deposition",["Soil1"]],
        ["Ca total deposition",["Soil1"]],
        ["Mg total deposition",["Soil1"]],
        ["K total deposition",["Soil1"]],
        ["Na total deposition",["Soil1"]],
        ["Effective precipitation",[]],
        ["Runoff",[]],
        ["Ca(2+) ionic concentration",["Stream"]],
        ["Mg(2+) ionic concentration",["Stream"]],
        ["Na(+) ionic concentration",["Stream"]],
        ["K(+) ionic concentration",["Stream"]],
        ["Sum of base cation concentrations (Ca + Mg + Na + K)",["Stream"]],
        ["SO4(2-) ionic concentration",["Stream"]],
        ["Cl(-) ionic concentration",["Stream"]],
        ["NO3(-) ionic concentration",["Stream"]],
        ["Sum of acid anion concentrations (SO4 + Cl + NO3 + F)",["Stream"]],
        ["H(+) ionic concentration",["Stream"]],
        ["pH",["Stream"]],
        ["HCO3 (Bicarbonate) ionic concentration",["Stream"]],
        ["Charge balance alkalinity",["Stream"]],
        ["Al(3+) ionic concentration",["Stream"]],
        ["Total Al(3+) charges",["Stream"]],
        ["Total aluminum in solution (ionic + SO4-F-DOC complexes)",["Stream"]],
        ["Aluminum in solution as organic complexes (AlA, Al(HA)(+)",["Stream"]],
        ["Total anionic charge from DOC (triprotic acid) in solution (H2AM, HA2M, A3M)",["Stream"]],
        ["Organic acid concentration",["Stream"]],
        ["Exchangeable Ca on soil as % of CEC",["Soil1"]],
        ["Exchangeable Mg on soil as % of CEC",["Soil1"]],
        ["Exchangeable Na on soil as % of CEC",["Soil1"]],
        ["Exchangeable K on soil as % of CEC",["Soil1"]],
        ["Base saturation of soil (ECa + EMg + ENa + EK)",["Soil1"]]
        ]
# Exporting results to dataframe
results = cu.get_result_dataframe(dataset, res_list)

# Verifying that future changes in AirTemperature, Precip and Discharge are correct
# Getting timeseries
temperature = cu.get_input_dataframe(dataset, [["Air temperature" , []]])
discharge = results.loc[:,'Runoff []']
precip = results.loc[:,'Effective precipitation []']

# Defining date masks, reference versus future
ref_mask = (results.index > '1970-12-01') & (results.index < '2001-01-01')
fut_mask = (results.index > '2070-12-01') & (results.index < '2101-01-01')

# Printing ratio increase and absolute increase
mean_Q_ref = np.mean(discharge.loc[ref_mask])
mean_Q_fut = np.mean(discharge.loc[fut_mask])
mean_P_ref = np.mean(precip.loc[ref_mask])
mean_P_fut = np.mean(precip.loc[fut_mask])
mean_T_ref = np.mean(temperature.loc[ref_mask])
mean_T_fut = np.mean(temperature.loc[fut_mask])
print('Ratio increase in discharge is: ',mean_Q_fut/ mean_Q_ref)
print('Ratio increase in Precipitation is: ',mean_P_fut / mean_P_ref)
print('Increase in AirTemperature is: +', mean_T_fut - mean_T_ref)

# Getting pH timeseries
saturation = results.loc[:,'pH [Stream]']

# Defining pre-industrial and future date masks
mask_past = (saturation.index < '1861-01-01')
mask_future = (saturation.index > '2089-12-01') & (saturation.index < '2101-01-01')

#Printing pre-industrial and future pH
past_sat = np.mean(saturation.loc[mask_past])
future_sat = np.mean(saturation.loc[mask_future])
print('Pre-industrial pH was: ',past_sat)
print('Future pH is: ',future_sat)

# Saving results (without weathering increase)
results.to_csv('Results_run_'+scenar+'_dQ_'+"{:.3f}".format(mean_Q_fut/ mean_Q_ref)+'_dP_'+"{:.3f}".format(mean_P_fut / mean_P_ref)+'_dT_'+"{:.3f}".format(mean_T_fut - mean_T_ref)+'.csv')    

# Loop to determine weathering increase needed for pH criteria: "future pH equals pre-industrial pH"
while past_sat >= future_sat:
        dataset.set_parameter_double("Yearly weathering increase", ['Soil1'], weaIn)
        dataset.run_model()  # to have some initial result values for comparison
        results = cu.get_result_dataframe(dataset, res_list)
        #saturation = results.loc[:,'Base saturation of soil (ECa + EMg + ENa + EK) [Soil1]']
        saturation = results.loc[:,'pH [Stream]']
        past_sat = np.mean(saturation.loc[mask_past])
        future_sat = np.mean(saturation.loc[mask_future])
        weaIn += 0.0001

# Saving results for weathering increase related to pH criteria
results_wea = cu.get_result_dataframe(dataset, res_list)
results_wea.to_csv('Results_run_'+scenar+'_dQ_'+"{:.3f}".format(mean_Q_fut/ mean_Q_ref)+'_weatheringIncrease_pH_'+"{:.4f}".format(weaIn)+'.csv')

# Re-running MAGIC with no weathering increase
weaIn = 0
dataset.set_parameter_double("Yearly weathering increase", ['Soil1'], weaIn)
dataset.run_model()

# Exporting results
results = cu.get_result_dataframe(dataset, res_list)

#Printing pre-industrial and future Soil Base Saturation
saturation = results.loc[:,'Base saturation of soil (ECa + EMg + ENa + EK) [Soil1]']
mask_past = (saturation.index < '1861-01-01')
mask_future = (saturation.index > '2089-12-01') & (saturation.index < '2101-01-01')
past_sat = np.mean(saturation.loc[mask_past])
future_sat = np.mean(saturation.loc[mask_future])
print('Pre-industrial Base saturation of soil was: ',past_sat)
print('Future Base saturation of soil is: ',future_sat)

# Loop to determine weathering increase needed for Soil Base Saturation criteria: "future Soil Base saturation equals pre-industrial Soil Base Saturation"
while past_sat >= future_sat:
        dataset.set_parameter_double("Yearly weathering increase", ['Soil1'], weaIn)
        dataset.run_model()  # to have some initial result values for comparison
        results = cu.get_result_dataframe(dataset, res_list)
        saturation = results.loc[:,'Base saturation of soil (ECa + EMg + ENa + EK) [Soil1]']
        #saturation = results.loc[:,'Charge balance alkalinity [Stream]']
        past_sat = np.mean(saturation.loc[mask_past])
        future_sat = np.mean(saturation.loc[mask_future])
        weaIn += 0.0001

# Saving results for weathering increase related to Soil Base Saturation criteria
results_wea2 = cu.get_result_dataframe(dataset, res_list)
results_wea2.to_csv('Results_run_'+scenar+'_dQ_'+"{:.3f}".format(mean_Q_fut/ mean_Q_ref)+'_weatheringIncrease_BS_'+"{:.4f}".format(weaIn)+'.csv')

# Re-running MAGIC with no weathering increase
weaIn = 0
dataset.set_parameter_double("Yearly weathering increase", ['Soil1'], weaIn)
dataset.run_model()

# Exporting results
results = cu.get_result_dataframe(dataset, res_list)

#Printing pre-industrial and future ANC
saturation = results.loc[:,'Charge balance alkalinity [Stream]']
mask_past = (saturation.index < '1861-01-01')
mask_future = (saturation.index > '2089-12-01') & (saturation.index < '2101-01-01')
past_sat = np.mean(saturation.loc[mask_past])
future_sat = np.mean(saturation.loc[mask_future])
print('Pre-industrial ANC was: ',past_sat)
print('Future ANC is: ',future_sat)

# Loop to determine weathering increase needed for ANC criteria: "future ANC equals pre-industrial ANC"
while past_sat >= future_sat:
        dataset.set_parameter_double("Yearly weathering increase", ['Soil1'], weaIn)
        dataset.run_model()  # to have some initial result values for comparison
        results = cu.get_result_dataframe(dataset, res_list)
        saturation = results.loc[:,'Charge balance alkalinity [Stream]']
        #saturation = results.loc[:,'Charge balance alkalinity [Stream]']
        past_sat = np.mean(saturation.loc[mask_past])
        future_sat = np.mean(saturation.loc[mask_future])
        weaIn += 0.0001

# Saving results for weathering increase related to ANC criteria
results_wea3 = cu.get_result_dataframe(dataset, res_list)
results_wea3.to_csv('Results_run_'+scenar+'_dQ_'+"{:.3f}".format(mean_Q_fut/ mean_Q_ref)+'_weatheringIncrease_ANC_'+"{:.4f}".format(weaIn)+'.csv')


Ratio increase in discharge is:  1.227866386458456
Ratio increase in Precipitation is:  1.2352609232319536
Increase in AirTemperature is:  4.189682872926783
Pre-industrial pH was:  5.371363636363627
Future pH is:  5.123484848484846
Pre-industrial Base saturation of soil was:  23.06308544122563
Future Base saturation of soil is:  17.911605897880044
Pre-industrial ANC was:  73.34916500454253
Future ANC is:  61.19844232061417
